In [86]:
import re
import glob
import hashlib
from tqdm import tqdm, trange

from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

In [87]:
EMPTY = set()
state = None

def add(v1, v2):
    return [x ^^ y for x, y in zip(v1, v2)]

def int64_to_vec(x):
    return vector(GF(2), [(x >> i) & 1 for i in range(64)])

def vec_to_int64(x):
    return sum(ZZ(x.get(i)) << i for i in range(64))

def _lshift_vec(vec, k):
    return [EMPTY] * k + list(vec)[:-k]

def _rshift_vec(vec, k):
    return list(vec)[k:] + [EMPTY] * k

def _mask_mask(vec, mask):
    _mask = int64_to_vec(mask)
    return [vec[i] if k else EMPTY for i, k in enumerate(_mask)]

def _mask_const(term, const):
    _mask = int64_to_vec(const)
    return [term if k else EMPTY for k in _mask]

def _process_vec(y):
    y = add(y, _mask_mask(_rshift_vec(y, 29), 0x5555555555555555))
    y = add(y, _mask_mask(_lshift_vec(y, 17), 0x71d67fffeda60000))
    y = add(y, _mask_mask(_lshift_vec(y, 37), 0xfff7eee000000000))
    y = add(y, _rshift_vec(y, 43))
    return y

In [88]:
const = 0xb5026f5aa96619e9

In [89]:
def twist():
    global state
    print("[+] twist()", flush=True)
    for i in range(312):
        # print("[*]", state[i], state[(i + 1) % 312], flush=True)
        # y = _mask_mask(state[(i + 1) % 312], lower) + _mask_mask(state[i], upper)
        state[i] = state[(i + 1) % 312][1:31] + state[i][31:] + [EMPTY]
        state[i] = add(state[i], state[(i + 156) % 312])
        state[i] = add(state[i], _mask_const(state[(i + 1) % 312][0], const))

In [90]:
print("Loading data", flush=True)
with open("public/output.txt", "r") as fin:
    ct = bytes.fromhex(fin.readline().split(" = ")[1].strip())
    iv = bytes.fromhex(fin.readline().split(" = ")[1].strip())
    _iv = int.from_bytes(iv, "big")
bits = [None] * 64 + list(map(int, bin(_iv)[2:].zfill(20000 - 64)[::-1]))
print("bits:", bits[:100], flush=True)

Loading data
bits: [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]


In [91]:
# correct_state = [69, 14850285750109030178, 2433108913545666511, 12165442406605326438, 16492750041495246232, 5985953121957622084, 14134967652723287079, 6469697030762821211, 12273613955114562522, 17259661581098255617, 16973225129691209572, 1671298410160447014, 2766666917229815482, 18152002698090042559, 6109896619580762394, 4424171998734786254, 14659121728155851846, 4021794363582100274, 1190070720064998876, 17113031305806140351, 7820767439185607712, 15761607804592599266, 10763736708310709923, 3652440110184888676, 16291849719008916140, 13531546450086099420, 10689944842290892320, 17550741367948958229, 5049524925867114490, 9085915584417350716, 2324128091746255319, 16182223945900014570, 12816883904370089749, 6338495201003790380, 690019768251267851, 11001203236598502930, 14483016109909577972, 3184981331538216080, 1953735306623729014, 3195912717810554853, 6727899073777700457, 14381881870900469873, 13948313970526077492, 1898363474447101142, 10158150047307123658, 3289353210223069781, 2924631213903838239, 12016550281043489442, 5288037825798194768, 1928511066588121966, 4262384579588166792, 8303760917298254875, 12691505544580951750, 10256275383962973353, 3231843563599501125, 14248865832675791448, 6248696301809803063, 13789104089182029495, 5843607658742135051, 17737777857298791677, 1619724124746087138, 14726661911679743991, 9857656590077387298, 6180484366290558943, 10323036535346379846, 15793517081317299253, 1266408713076512704, 16933434268416011523, 17261828307174336836, 13654271599404317376, 13402228851820876384, 2849994893206821249, 12151288158736186101, 1169145218531206836, 741819289445341678, 7734691436031090977, 11798452477212373740, 7154217834858219043, 17106406639752114760, 9038218857931263102, 17629303477546251171, 9561911214626190193, 5819230377587513481, 17692394905852954683, 16584678928098916908, 9552629934797441944, 1960578554505310312, 5314126755454229151, 10420019664327839774, 1920579974014384453, 11814572504873913467, 6813620197529319584, 14530818300613956521, 5717960335638514495, 1826089317315542084, 9746540325693920339, 12293903579751661981, 12072023852027505236, 4158161956491320192, 13187797178028234211, 10347220378338392049, 4109685635204129564, 17905396678924636242, 12816169692784168356, 3122202157254754454, 15049344592201270471, 11743069746039313118, 5518361885899535639, 5523064766773051210, 13099261390903694748, 15735208550749564724, 4954808013047354906, 8530172943937275951, 6039486853166908039, 17250917600359853056, 16884230491275901434, 7146627352729827385, 5377079253085891149, 11325711945158075346, 1175841450242341895, 3536321062507234995, 17987804153441270512, 13005249981618285105, 788148126685329522, 11570339277811300998, 18165650536299414961, 18232708859827982024, 16349646165263929134, 12066432301769178729, 6487193457298837072, 229428936786228159, 18318805619883399702, 7278903377176379701, 354652449667105449, 5081617143372943163, 7497558559813792697, 4724303652352827360, 14904849903329294870, 9654616591528403259, 7868294853033500304, 18377173560417505033, 6775725803195226959, 8023923270416842564, 16062558331775015600, 12622106697264283399, 16791480129290997618, 9642624528887989871, 5568699880367158460, 2292132947419875533, 2860843525364350878, 15044875604876382556, 2322206644710534986, 7994086170589054618, 18221232115843518168, 17932509913338872345, 16745957960550270253, 3041547535235218354, 6052294403885337831, 14097508762821320460, 7722979236945438530, 5412312762183033959, 4330257397020393615, 14066425088243174085, 5908415129829941105, 14613827790321653588, 10392847639868851184, 17319939956249493552, 5471647358638282398, 4323805273413622683, 45673835583717352, 2768951858098230386, 10533333615840814773, 5792976824891660247, 6760470939962645323, 12059043970688713392, 1141514292704724985, 14376006554387754613, 7848659205171623279, 2102102900942141192, 3226835475814606107, 14939040302529096563, 4886941580894909285, 16293689027031803210, 15262468573549282700, 10155481829793770459, 17540686337364843742, 9359123964020029075, 14341050618119309112, 12507168617459174171, 18130902956048423714, 13665444396961473675, 15046176000598795220, 17555217830165395083, 13911393252134719145, 7231927861169253028, 9656800224326818500, 16619073847145426834, 9362500945733192770, 16248043795163080710, 17119492047852061864, 3771777233785612247, 5066099564990360212, 6528217348327614459, 14557732355667791293, 3983967235049499954, 14972865518559772823, 18309892963862300370, 2702565340066128524, 14581826585510807404, 6785923105704665172, 3178783832504015555, 4487329387126092314, 14607659555773674854, 14611690777913540246, 8650469441940155399, 16395201898136500197, 17054554322277903942, 8808727791174901242, 4659912545766869753, 16280093529780582003, 8227777100883485324, 6391648914081892006, 8086830636050859321, 3036329342931530679, 9550803986416279819, 15723715638974760566, 3600131081381178995, 4213008057754089498, 7800227656146748278, 16093206001476543952, 34092242650846205, 10796394235992786784, 3737507590365686562, 4635293704024224739, 6248356048367582628, 58651963418898924, 7311945926270131048, 14172021054058588514, 786669623303546363, 13707605466632827662, 12652037425211239948, 3614621764645931879, 14219218441530235661, 17705085596277634153, 10999205772563189142, 14725863541120225273, 16572032274137847272, 7422663040433123398, 1367429326763685491, 18188880427089780784, 3020113589899409137, 17861326137394640984, 12811521970281811451, 9930017077979937986, 14010209903230004926, 14244903658037443384, 5104596873780686687, 6541450802991281799, 11274015481877303440, 9263644877386569901, 2094508980097214663, 9265217547056391424, 12891725173846409312, 7368085360949604417, 17218569647880682568, 971578709965983544, 18068295194496415202, 11447673531710052504, 1050990749156555294, 6617961706988870227, 8573699778506182520, 18351382407965474900, 11818212571937159515, 13279942519919393974, 2917195483987343542, 5267380285004415761, 6838848714000104164, 17641263518300288598, 2215745899135842567, 12801347909799411794, 10290562142964170536, 7914162112720597371, 887945320632678796, 7498884365341905591, 6837169369110795546, 12081337382906878940, 10170622662080121124, 4164483125943285453, 16714749428117493289, 6397906319726017667, 17269200682146128380, 7643408118607526902, 13606395898320721039, 3995636887459959022, 2399445660061961468, 3356471279068499315, 17457407818660850527, 8387841308984493141, 9827685184581644270, 122379904170852775, 6361209092488635527, 10367030262445106875, 6815142494923282099, 12700609328551660665, 12995076509576312783, 4314859466640917306, 7425591798559758436, 4258930916333954548, 12756686654123292952, 4264489197639561415, 15190066733707627825, 2083158051121733889]
# correct_coef = [[(correct_state[i] >> j) & 1 for j in range(64)] for i in range(len(correct_state))]
# correct_coef = sum(correct_coef, [])
# correct_coef = vector(GF(2), correct_coef)

In [92]:
print("Initialising variables", flush=True)
state = [[{i * 64 + j} for j in range(64)] for i in range(312)]

res_mat = Matrix(GF(2), 64, 312 * 64)
coef_mat = Matrix(GF(2), len(bits) - 64, 312 * 64)
rhs = vector(GF(2), len(bits) - 64, bits[64:])

Initialising variables


In [93]:
print("Calculating coefficients", flush=True)
idx = 0
for i in range(len(bits)):
# for i in range(1000):
    if idx == 0:
        twist()

    y = state[idx]
    idx = (idx + 1) % 312

    y = _process_vec(y)
    msb = y[63]

    if bits[i] is not None:
        # calculated = sum(correct_coef[k] for k in msb) % 2
        # print(f"[{i:<5}]", calculated, "==", bits[i], flush=True)
        # assert calculated == bits[i]

        if idx == 0:
            msb_s = str(msb)
            print(f"[{i:<5}] {msb_s[:20]}...{msb_s[-20:]} == {bits[i]}", flush=True)
    
    if bits[i] is not None:
        # extract coefficients from msb
        for k in msb:
            coef_mat[i - 64, k] = 1
    else:
        for k in msb:
            res_mat[i, k] = 1

print("Done!")

Calculating coefficients
[+] twist()
[311  ] {65, 19960, 19913, 7...6, 19930, 92, 19967} == 0
[+] twist()
[623  ] {9984, 10049, 130, 6..., 10076, 157, 19967} == 0
[+] twist()
[935  ] {128, 129, 130, 9976... 19960, 9978, 19967} == 1
[+] twist()
[1247 ] {10112, 257, 10113, ...19961, 19963, 19967} == 1
[+] twist()
[1559 ] {10112, 10113, 10114...19963, 19964, 19967} == 1
[+] twist()
[1871 ] {10240, 10241, 10242... 9980, 19965, 19967} == 1
[+] twist()
[2183 ] {256, 257, 258, 387,..., 9978, 9982, 19967} == 1
[+] twist()
[2495 ] {10240, 10241, 258, ...19961, 19963, 19960} == 1
[+] twist()
[2807 ] {9984, 513, 10240, 1..., 9978, 19963, 9980} == 0
[+] twist()
[3119 ] {10240, 513, 10242, ...19961, 19964, 19965} == 0
[+] twist()
[3431 ] {10240, 10241, 10242...19960, 19962, 19966} == 0
[+] twist()
[3743 ] {512, 513, 10242, 10...19961, 19963, 19967} == 1
[+] twist()
[4055 ] {576, 10305, 577, 10...19963, 19964, 19967} == 1
[+] twist()
[4367 ] {10240, 10241, 10242...19964, 19965, 19967} == 1
[+] twist

In [94]:
# print("TESTING")
# correct_recover = ''.join(map(str, rhs[64:1000]))
# our_recover = ''.join(map(str, (coef_mat * correct_coef)[64:1000]))
# print("Correct:", correct_recover)
# print(" We got:", our_recover)
# assert correct_recover == our_recover

TESTING
Correct: 110110000001100000011111000001101101101101110000001011100010000110111111011001001110111000100111101010101000011100010000111111100001000001100110101101001001010110001100001011110111011010101111011010000100011011101100101010111100010111011110100001101011111111011111011010011110100110101100101111101101000010001100100010110101110100010000010001001000111110011011101111000011010000010100101000101111010010000100000110011010001100101101001110100111111111001110111111110110111100011011101100010010100011011001100001011010111110101011100001010110010000011001010110001010110100101100100001101011110101000011000100110000111110000110001111010000110011111011001000101001010101101100001110110010000100010111010111101101010000010110110000110110111111000101100110011110101101110011001000000101110100100000110000001111101110101111000011001110000101000110011110111101100000001100110001111110011000100001101010010101000001011011111110000000001001000011
 We got: 0111010100001111001010000011110010110

AssertionError: 

In [95]:
print("Started solving equations...", flush=True)

res = coef_mat.solve_right(rhs)
print("solution:", res, flush=True)

ker = coef_mat.right_kernel()
print("kernel:", ker, flush=True)

Started solving equations...
solution: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 

kernel: Vector space of degree 19968 and dimension 32 over Finite Field of size 2
Basis matrix:
32 x 19968 dense matrix over Finite Field of size 2


In [96]:
print(coef_mat.dimensions())

(19936, 19968)


In [97]:
basis = ker.basis()
for vec in basis:
    s = ''.join(map(str, vec))
    print(s[:20], s[-20:], s.count('1'))

10000000000000000000 00000000000000000000 1
01000000000000000000 00000000000000000000 1
00100000000000000000 00000000000000000000 1
00010000000000000000 00000000000000000000 1
00001000000000000000 00000000000000000000 1
00000100000000000000 00000000000000000000 1
00000010000000000000 00000000000000000000 1
00000001000000000000 00000000000000000000 1
00000000100000000000 00000000000000000000 1
00000000010000000000 00000000000000000000 1
00000000001000000000 00000000000000000000 1
00000000000100000000 00000000000000000000 1
00000000000010000000 00000000000000000000 1
00000000000001000000 00000000000000000000 1
00000000000000100000 00000000000000000000 1
00000000000000010000 00000000000000000000 1
00000000000000001000 00000000000000000000 1
00000000000000000100 00000000000000000000 1
00000000000000000010 00000000000000000000 1
00000000000000000001 00000000000000000000 1
00000000000000000000 00000000000000000000 1
00000000000000000000 00000000000000000000 1
00000000000000000000 00000000000

In [98]:
required = set()
for vec in res_mat:
    for idx in range(312 * 64):
        if vec[idx] == 1:
            required.add(idx)

In [99]:
for vec in res_mat:
    print([idx for idx in range(312 * 64) if vec[idx] == 1])

[56, 64, 74, 91, 9993, 10010, 10039, 10047]
[120, 128, 138, 155, 10057, 10074, 10103, 10111]
[184, 192, 202, 219, 10121, 10138, 10167, 10175]
[248, 256, 266, 283, 10185, 10202, 10231, 10239]
[312, 320, 330, 347, 10249, 10266, 10295, 10303]
[376, 384, 394, 411, 10313, 10330, 10359, 10367]
[440, 448, 458, 475, 10377, 10394, 10423, 10431]
[504, 512, 522, 539, 10441, 10458, 10487, 10495]
[568, 576, 586, 603, 10505, 10522, 10551, 10559]
[632, 640, 650, 667, 10569, 10586, 10615, 10623]
[696, 704, 714, 731, 10633, 10650, 10679, 10687]
[760, 768, 778, 795, 10697, 10714, 10743, 10751]
[824, 832, 842, 859, 10761, 10778, 10807, 10815]
[888, 896, 906, 923, 10825, 10842, 10871, 10879]
[952, 960, 970, 987, 10889, 10906, 10935, 10943]
[1016, 1024, 1034, 1051, 10953, 10970, 10999, 11007]
[1080, 1088, 1098, 1115, 11017, 11034, 11063, 11071]
[1144, 1152, 1162, 1179, 11081, 11098, 11127, 11135]
[1208, 1216, 1226, 1243, 11145, 11162, 11191, 11199]
[1272, 1280, 1290, 1307, 11209, 11226, 11255, 11263]
[1336

In [100]:
basis = list(basis)
basis = [vec for vec in basis if len(set(idx for idx in range(len(vec)) if vec[idx]) & required) > 0]
for vec in basis:
    s = ''.join(map(str, vec))
    print(s[:20], s[-20:], s.count('1'))

00000000000000000000 01100111000000110110 9907


In [101]:
def AES_decrypt(key, iv, ct):
    key = hashlib.md5(key).digest()
    iv = hashlib.md5(iv).digest()
    cipher = AES.new(key=key, iv=iv, mode=AES.MODE_CBC)
    msg = cipher.decrypt(ct)
    try:
        return unpad(msg, 16).decode()
    except ValueError:
        return

for _null in span(basis):
    _sol = res + _null
    _recovered = res_mat * _sol
    key = ''.join(map(str, _recovered))[::-1]
    key = int(key, 2).to_bytes(len(key) // 8, 'big')
    print(AES_decrypt(key, iv, ct))

None
flag{w3_d0n7_know_do3s_g0D_p14y_d1c3_w1th_th3_un1ver53_8u7_3e_k0n3_4t_13457_c0mpu73rs_d07}
